# Tasks 2020
### [ Machine Learning and Statistics Module GMIT 2020 ]
----

These are my solutions to the Task assessment.

Completed by: *Olga Rozhdestvina (G00387844@gmit.ie)*

----
- __TASK 1__: Calculate square root of 2 to 100 decimal places

To calculate square root of 2 we can use Babylonian method - it is the trial and error method by reapeating calculations until the success occurs.[1] [2]

1. https://en.wikipedia.org/wiki/Methods_of_computing_square_roots#Babylonian_method
2. https://www.geeksforgeeks.org/square-root-of-a-perfect-square/

In [1]:
def sqRoot(n=2):
    """
    Function that calculates square root of 2 to 100 decimal places
    """
    x = n
    # Initial guess for the square root based
    y = 1
    # Set accuracy to draw a line for acceptable errors
    accuracy = 0.000000001
    # Loop until we are happy with the accuracy
    while x - y > accuracy:
        # Calculate a better guess for the square root
        x = (x+y)/2
        y = n/x
    # Formatting to get 100 digits (though it is not working)
    return '{:.100}'.format(x)
    

In [2]:
sqRoot(2)

'1.4142135623746898698271934335934929549694061279296875'